   #                    stylometric Features Extraction 

##  <font color="blue">importing required libraries</font>

In [1]:
import collections as coll
import math
import pickle
import string
import numpy as np
from nltk.corpus import cmudict
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
cmuDictionary = None
from os import  path

import pandas as pd
import glob

import warnings
warnings.filterwarnings("ignore")
import math


import io, os, sys, types
import IPython
import pandas as pd
import seaborn as sns
import nltk
import glob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")

from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from scipy import sparse
import math
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import operator
import itertools
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression

In [2]:
stop_words_list="C:/Users/RAVIKUMAR/PycharmProjects/Authorship_Attribution_Instance/Data_2/new_stop_words.txt"
from nltk.tokenize import sent_tokenize
import nltk
import numpy as np
from sklearn.preprocessing import StandardScaler
# removing stop words plus punctuation.

In [3]:
with open(stop_words_list, encoding='utf-8') as file:
    reader=file.read()
    reader = [reader.split()]
    stopword = sum(reader, [])
#ss =".,"    
def text_process(text):
    nopunct = "".join([char for char in text if char not in string.punctuation])
    tokens = nltk.word_tokenize(nopunct)
    nopunct = " ".join([word for word in tokens if word not in stopword])
    return nopunct

In [4]:
def Avg_wordLength(str):
    norm=[]
    str.translate(string.punctuation)
    tokens = word_tokenize(str)
    st = [",", ".", "'", "!", '"', "#", "$", "%", "&", "(", ")", "*", "+", "-", ".", "/", ":", ";", "<", "=", '>', "?",
          "@", "[", "\\", "]", "^", "_", '`', "{", "|", "}", '~', '\t', '\n']
    words = [word for word in tokens if word not in stopword and word not in st]
 
    return np.average([len(word) for word in words])


# ----------------------------------------------------------------------------


# returns avg number of characters in a sentence
def Avg_SentLenghtByCh(text):
    tokens = sent_tokenize(text)
    return np.average([len(token) for token in tokens])


# ----------------------------------------------------------------------------

# returns avg number of words in a sentence
def Avg_SentLenghtByWord(text):
    tokens = sent_tokenize(text)
    return np.average([len(token.split()) for token in tokens])



# -----------------------------------------------------------------------------

# COUNTS SPECIAL CHARACTERS NORMALIZED OVER LENGTH OF CHUNK
def CountSpecialCharacter(text):
    st = ["#", "$", "%", "&", "(", ")", "*", "+", "-", "/", "<", "=", '>',
          "@", "[", "\\", "]", "^", "_", '`', "{", "|", "}", '~', '\t', '\n']
    count = 0
    for i in text:
        if (i in st):
            count = count + 1
    return count / len(text)


# ----------------------------------------------------------------------------

def CountPuncuation(text):
    st = [",", ".", "'", "!", '"', ";", "?", ":", ";"]
    count = 0
    for i in text:
        if (i in st):
            count = count + 1
    return float(count) / float(len(text))


# ----------------------------------------------------------------------------


# also returns Honore Measure R
def hapaxLegemena(text):
    words = RemoveSpecialCHs(text)
    V1 = 0
    # dictionary comprehension . har word kay against value 0 kardi
    freqs = {key: 0 for key in words}
    for word in words:
        freqs[word] += 1
    for word in freqs:
        if freqs[word] == 1:
            V1 += 1
    N = len(words)
    V = float(len(set(words)))
    if(N>0):
        R = 100 * math.log(N) / max(1, (1 - (V1 / V)))
        h = V1 / N
    else:
        R=0.00
    return R


# ---------------------------------------------------------------------------

def hapaxDisLegemena(text):
    words = RemoveSpecialCHs(text)
    count = 0
    # Collections as coll Counter takes an iterable collapse duplicate and counts as
    # a dictionary how many equivelant items has been entered
    freqs = coll.Counter()
    freqs.update(words)
    for word in freqs:
        if freqs[word] == 2:
            count += 1
    try:
        h = count / float(len(words))
        S = count / float(len(set(words)))
        t = S+h
    except:
        t=0.00
    return t


# ---------------------------------------------------------------------------

# c(w)  = ceil (log2 (f(w*)/f(w))) f(w*) frequency of most commonly used words f(w) frequency of word w
# measure of vocabulary richness and connected to zipfs law, f(w*) const rak kay zips law say rank nikal rahay hein
def AvgWordFrequencyClass(text):
    words = RemoveSpecialCHs(text)
    # dictionary comprehension . har word kay against value 0 kardi
    freqs = {key: 0 for key in words}
    for word in words:
        freqs[word] += 1
    try:
        maximum = float(max(list(freqs.values())))
    except:
        maximum = 0.00
    return np.average([math.floor(math.log((maximum + 1) / (freqs[word]) + 1, 2)) for word in words])


# --------------------------------------------------------------------------
# TYPE TOKEN RATIO NO OF DIFFERENT WORDS / NO OF WORDS
def typeTokenRatio(text):
    words = word_tokenize(text)
    return len(set(words)) / len(words)


# --------------------------------------------------------------------------
# logW = V-a/log(N)
# N = total words , V = vocabulary richness (unique words) ,  a=0.17
# we can convert into log because we are only comparing different texts
def BrunetsMeasureW(text):
    words = RemoveSpecialCHs(text)
    a = 0.17
    V = float(len(set(words)))
    N = len(words)
    try:
        B = (V - a) / (math.log(N))
    except:
        B=0.00
        
    return B


# ------------------------------------------------------------------------
def RemoveSpecialCHs(text):
    text = word_tokenize(text)
    st = [",", ".", "'", "!", '"', "#", "$", "%", "&", "(", ")", "*", "+", "-", ".", "/", ":", ";", "<", "=", '>', "?",
          "@", "[", "\\", "]", "^", "_", '`', "{", "|", "}", '~', '\t', '\n']

    words = [word for word in text if word not in st]
    return words


# -------------------------------------------------------------------------
# K  10,000 * (M - N) / N**2
# , where M  Sigma i**2 * Vi.
def YulesCharacteristicK(text):
    words = RemoveSpecialCHs(text)
    N = len(words)
    freqs = coll.Counter()
    freqs.update(words)
    vi = coll.Counter()
    vi.update(freqs.values())
    M = sum([(value * value) * vi[value] for key, value in freqs.items()])
    try:
        K = 10000 * (M - N) / math.pow(N, 2)
    except:
        K = 0.00
    return K


# -------------------------------------------------------------------------


# -1*sigma(pi*lnpi)
# Shannon and sympsons index are basically diversity indices for any community
def ShannonEntropy(text):
    words = RemoveSpecialCHs(text)
    lenght = len(words)
    freqs = coll.Counter()
    freqs.update(words)
    arr = np.array(list(freqs.values()))
    distribution = 1. * arr
    distribution /= max(1, lenght)
    import scipy as sc
    H = sc.stats.entropy(distribution, base=2)
    # H = sum([(i/lenght)*math.log(i/lenght,math.e) for i in freqs.values()])
    return H


# ------------------------------------------------------------------
# 1 - (sigma(n(n - 1))/N(N-1)
# N is total number of words
# n is the number of each type of word
def SimpsonsIndex(text):
    words = RemoveSpecialCHs(text)
    freqs = coll.Counter()
    freqs.update(words)
    N = len(words)
    n = sum([1.0 * i * (i - 1) for i in freqs.values()])
    try:
        D = 1 - (n / (N * (N - 1)))
    except:
        D = 0.00
    return D


def Each_sentence(sequence):
    sequence = sent_tokenize(sequence)
    return sequence



In [5]:
def FeatureExtration(text):
    # cmu dictionary for syllables
    df = pd.DataFrame()
    global cmuDictionary
    cmuDictionary = cmudict.dict()
    All_sentences_per_instance = Each_sentence(text)
    vector = []
    for line in All_sentences_per_instance:
        feature = []
        
        meanwl = (Avg_wordLength(line))
        feature.append(meanwl)
        
        meansl = (Avg_SentLenghtByCh(line))
        feature.append(meansl)
        
        mean = (Avg_SentLenghtByWord(line))
        feature.append(mean)
        
        means = CountSpecialCharacter(line)
        feature.append(means)
        
        p = CountPuncuation(line)
        feature.append(p)
        
        TTratio = typeTokenRatio(line)
        feature.append(TTratio)
        
        #-----------new feature adding-----------
        h = hapaxLegemena(line)
        feature.append(h)
        
        B = BrunetsMeasureW(line)
        feature.append(B)
        
        Y = YulesCharacteristicK(line)
        feature.append(Y)
        
        S = ShannonEntropy(line)
        feature.append(S)
        
        SI =SimpsonsIndex(line)
        feature.append(SI)
        
        
        AWF = AvgWordFrequencyClass(line)
        feature.append(AWF)
        
        h = hapaxDisLegemena(line)
        feature.append(h)
        
        vector.append(feature)
    return vector



In [6]:
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
all_instances={
'hrudayashiva':["01","02","03","04","05","06","07","08","09",10,11,12],
    'ravibeliger':[13,14,15,16,17,18,19,20,21,22,23,24,25],
    'somashaker':[26,27,28,29,30,31,32,33,34,35,36],
    'chandrashekark':[37,38,39,40,41,42,43,44,45,46,47],
    'usha':[48,49,50,51,52,53,54,55,56,57,58,59,60],
    'bhagavan':[61,62,63,64,65,66,67,68,69,70,71,72],
    'NaDisoza':[73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92],
    'srinatha':[93,94,95,96,97,98,99,100]
}

# stop_words_list = "C://Users/RAVIKUMAR/PycharmProjects/Authorship_Attribution_Instance/Data_2/new_stop_words.txt"


instance_by_author={}
for author,AllFiles_Per_author in all_instances.items():
    for i in AllFiles_Per_author:
        for name in glob.glob(f"C://Users/RAVIKUMAR/PycharmProjects/Authorship_Attribution_Instance/Data_2/train/instance{i}.txt"):
            with open(name, encoding='utf-8') as file:
                instance_by_author[author+str(i)]=file.read()
all_authors=[]
text=[]
original = pd.DataFrame()
for author, file in instance_by_author.items():
    all_authors.append(author)
    text.append(file)
    
original["Authors with id"] = all_authors
original["text files"]=text
df=pd.DataFrame()  

In [7]:
X=original["text files"]
y=all_authors


In [8]:
print("------------------LEXICAL FEATURES EXTRACTING----------")
print()
author_fvs={}
feature_vectors=[]
for (text,author) in zip(X, y):
   # clean_text = text_process(text)
    #print(author, clean_text)
    author_fvs[author]=FeatureExtration(text)
    #feature_vectors.append(FeatureExtration(text))
    
print("successfully extracted all the features\n \n ")


------------------LEXICAL FEATURES EXTRACTING----------

successfully extracted all the features
 
 


In [9]:
# avgValues={}
# def column_sum(lst):   
#     return list(map(sum, zip(*lst)))
    
# for i, j in author_fvs.items():
#     avgValues[i] = column_sum(j)

In [13]:
train_data = pd.DataFrame()
train_data["X_train"] =X
train_data["y_train"] = y
train_data["features"] =author_fvs.values()


In [14]:
Y = pd.Series(train_data["y_train"])
Y.replace(r'(^ra.*)','Ravibeligeri',regex=True, inplace = True)
Y.replace(r'(^hr.*)','Hrudayashiva',regex=True, inplace = True)
Y.replace(r'(^som.*)','Somashaker',regex=True, inplace = True)
Y.replace(r'(^ch.*)', 'Chandrashekar_k', regex=True, inplace=True)
Y.replace(r'(^us.*)', 'Usha', regex=True, inplace=True)
Y.replace(r'(^bh.*)', 'Bhagavan', regex=True, inplace=True)
Y.replace(r'(^sri.*)', 'Srinatha', regex=True, inplace=True)
Y.replace(r'(^Na.*)', 'NaDisoza', regex=True, inplace=True)

In [15]:
train_data["y_train"] = Y
vectors = train_data["features"]
arr = [0,0,0,0,0,0,0,0,0,0,0,0,0]
for i in vectors:
    for k in range(2976-len(i)):
        i.append(arr)

In [16]:
vec = np.array(vectors)
# vec
size = []
for i in vectors:
        size.append(len(i))
train_data["size"] = size  


In [17]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

flatten_final=[]
for i in range(len(vectors)):
    flatten_final.append(flatten_list(vec[i]))

# nested_list = vectors[0]
# print('Original List', len(nested_list)
print('Transformed Flat List', len(flatten_final))

Transformed Flat List 100


In [18]:
flatten_final = np.asarray(flatten_final, )
a = flatten_final
col_mean = np.nanmean(a, axis=0 )
#Find indices that you need to replace
inds = np.where(np.isnan(a))
#Place column means in the indices. Align the arrays using take
a[inds] = np.take(col_mean, inds[1])
flatten_final = a
flatten_final=sparse.csr_matrix(flatten_final)

In [24]:
yy=train_data["y_train"]
X_train, X_test, y_train, y_test = train_test_split(flatten_final,yy,test_size=0.25, random_state=789)


In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

cmodel_SVM =SVC(kernel='linear')
cmodel_SVM = cmodel_SVM.fit(X_train, y_train)

print("SVM Result")
print("Taining Accuracy :",cmodel_SVM.score(X_train, y_train))
print("Testing Accuracy: ",cmodel_SVM.score(X_test, y_test))

predictions = cmodel_SVM.predict(X_test)
print(classification_report(y_test, predictions))


SVM Result
Taining Accuracy : 1.0
Testing Accuracy:  0.6
                 precision    recall  f1-score   support

       Bhagavan       0.50      0.50      0.50         2
Chandrashekar_k       0.40      0.50      0.44         4
   Hrudayashiva       0.75      1.00      0.86         3
       NaDisoza       0.80      1.00      0.89         4
   Ravibeligeri       0.50      0.20      0.29         5
     Somashaker       0.50      0.50      0.50         4
       Srinatha       1.00      0.50      0.67         2
           Usha       0.50      1.00      0.67         1

    avg / total       0.60      0.60      0.57        25



In [27]:
def fun_model(model):
    print(model)
    model = model.fit(X_train.toarray(), y_train)
    print("Taining Accuracy :",model.score(X_train.toarray(), y_train))
    print("Testing Accuracy: ",model.score(X_test.toarray(), y_test))
    predictions = model.predict(X_test.toarray())
    print(classification_report(y_test, predictions))

    
cmodel_NB = GaussianNB(priors=None)
cmodel_Deci  = DecisionTreeClassifier()
cmodel_Rand = RandomForestClassifier(max_depth=2, random_state=45)
cmodel = [ cmodel_NB,cmodel_Deci ,cmodel_Rand]
for i in cmodel:
    fun_model(i)

GaussianNB(priors=None)
Taining Accuracy : 0.49333333333333335
Testing Accuracy:  0.36
                 precision    recall  f1-score   support

       Bhagavan       0.20      0.50      0.29         2
Chandrashekar_k       0.33      0.50      0.40         4
   Hrudayashiva       0.43      1.00      0.60         3
       NaDisoza       0.00      0.00      0.00         4
   Ravibeligeri       0.00      0.00      0.00         5
     Somashaker       0.40      0.50      0.44         4
       Srinatha       0.50      0.50      0.50         2
           Usha       0.00      0.00      0.00         1

    avg / total       0.22      0.36      0.27        25

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitte

In [ ]:

def Avg_wordLength(str):
    norm=[]
    str.translate(string.punctuation)
    tokens = word_tokenize(str)
    st = [",", ".", "'", "!", '"', "#", "$", "%", "&", "(", ")", "*", "+", "-", ".", "/", ":", ";", "<", "=", '>', "?",
          "@", "[", "\\", "]", "^", "_", '`', "{", "|", "}", '~', '\t', '\n']
    words = [word for word in tokens if word not in stopword and word not in st]
    print(words)
    for i in words: 
        print(len(i), end="           ")
    return np.average([len(word) for word in words])

s="ಹೋಳಿ ಹಬ್ಬದ ಸಂಭ್ರಮ,ಖುಷಿಗಳನ್ನು ನೀವೂ ಅನುಭವಿಸಿರುತ್ತೀರಿ."
Avg_wordLength(s)

In [ ]:
fs_all = ["Avg_wordLength","Avg_SentLenghtByCh","Avg_SentLenghtByWord","CountSpecialCharacter","CountPuncuation","typeTokenRatio",
         
         "hapaxLegemena","BrunetsMeasureW","YulesCharacteristicK","ShannonEntropy","SimpsonsIndex","AvgWordFrequencyClass","hapaxDisLegemena"
         ]


In [ ]:
fs_all_short = ["AwL","ASLBC","ASLBW","CSC","CP","ttR",
         
         "HL","BM","YC","SE","SI","AWFC","HD"
         ]


In [ ]:
for i in fs_all:
    print(i, end=" ")
print("\n ------------------------------------------------------------------------------------------------------------------------")
for i in fs_all_short:
    print(i, end="       ")
# print("\n\n")
c= 0
for i in avgValues.values():
    c+=1
    print("instance", c)
    for j in i:
        print("{:.2f}".format(j), end="    " )

In [ ]:
l